In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [5]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [6]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
y = train_data["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch", "Age","Fare"]

X = pd.get_dummies(train_data[features])
X_test =pd.get_dummies(test_data[features])


In [8]:
#pipeline

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

num_cols=[col for col in X if X[col].dtype in ['int64','float64'] ]
cat_cols=[col for col in X if X[col].dtype is 'object' ]


numerical_transformer = KNNImputer(n_neighbors=10)


categorical_transformer = KNNImputer(n_neighbors=10)


#('onehot', OneHotEncoder(handle_unknown='ignore',sparse='false')
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])

sample_space=[]

for var1 in sample_space:
    for var2 in sample_space:
        model_dummy = RandomForestClassifier(n_estimators=var1, max_leaf_nodes=var2, random_state=21)
        pip_dummy = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model_dummy', model_dummy)
                     ])
        pip_dummy.fit(X, y)
        scores=cross_val_score(pip_dummy,X,y,cv=5,scoring='accuracy')
        print("For leaves = {} and tress = {} the mean {} of scores is {}".format(var2,var1,scores,scores.mean()))
print(X.head())




   Pclass  SibSp  Parch   Age     Fare  Sex_female  Sex_male
0       3      1      0  22.0   7.2500           0         1
1       1      1      0  38.0  71.2833           1         0
2       3      0      0  26.0   7.9250           1         0
3       1      1      0  35.0  53.1000           1         0
4       3      0      0  35.0   8.0500           0         1


In [9]:
model = RandomForestClassifier(n_estimators=18, max_leaf_nodes=16, random_state=21)
pip = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
scores=cross_val_score(pip,X,y,cv=5,scoring='accuracy')
print("After analysis of CVS mean score we conclude that:")
print("Our best model is with 20 leaves and 58 trees  and the scores are {} with accuracy {}".format(scores,scores.mean()))

pip.fit(X, y)

predic = pip.predict(X_test)
print("OK")

After analysis of CVS mean score we conclude that:
Our best model is with 20 leaves and 58 trees  and the scores are [0.65363128 0.67977528 0.73595506 0.79213483 0.74719101] with accuracy 0.7217374929382965
OK


In [10]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predic})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
